<a href="https://colab.research.google.com/github/Abhi-213/Deep-learning/blob/main/Deployment_frontend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.8 MB/s eta 0:00:00


In [67]:
# Create a new file named `app.py`
with open('app.py', 'w') as f:
    f.write("""
import streamlit as st
import requests
import numpy as np
from PIL import Image
import io

# Set the title of the app
st.title('Metastasis Segmentation')

# File uploader for image input
uploaded_file = st.file_uploader("Upload an MRI image", type=["jpg", "jpeg", "png","tif","tiff"])

if uploaded_file is not None:
    # Display the uploaded image
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image', use_column_width=True)

    # Convert image to bytes for sending to the FastAPI backend
    image_bytes = io.BytesIO()
    image.save(image_bytes, format='PNG')
    image_bytes.seek(0)

    # Send a request to the FastAPI backend
    url = 'https://1794-34-31-174-252.ngrok-free.app/predict/attention_unet/'  # Replace with your ngrok URL
    files = {"file": ("image.png", image_bytes, "image/png")}

    response = requests.post(url, files=files)


    if response.status_code == 200:
        prediction = np.array(response.json()['prediction'])
        # The returned prediction should be 4D (1, 1, 256, 256), convert it to 2D (256, 256)
        mask = prediction[0][0]
        mask = (mask * 255).astype(np.uint8)
        st.write("Prediction:", np.array(mask))
        mask_image = Image.fromarray(mask)
        # Display the segmentation mask as an image
        st.image(mask_image, caption='Predicted Segmentation Mask', use_column_width=True)

    else:
        st.write("Error:", response.status_code, response.text)
""")


In [ ]:
!npx localtunnel --port 8501

your url is: https://long-rules-sink.loca.lt
^C


In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken 2nSGot1EZaMDocY77BT3REXM9Vb_5ZNM7Zz7Jw53fGBHQqWGx

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

ngrok.kill()
# Launch FastAPI app using ngrok
ngrok_tunnel = ngrok.connect(8501)
print('Public URL:', ngrok_tunnel.public_url)

!streamlit run app.py


Public URL: https://2f6b-34-125-183-66.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.183.66:8501

